In [ ]:
!python --version

Python 2.7.17


In [ ]:
!pip install zipline

In [ ]:
!apt-get install libatlas-base-dev python-dev gfortran pkg-config libfreetype6-dev hdf5-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
python-dev is already the newest version (2.7.15~rc1-1).
hdf5-tools is already the newest version (1.10.0-patch1+docs-4).
libatlas-base-dev is already the newest version (3.10.3-5).
gfortran is already the newest version (4:7.4.0-1ubuntu2.3).
libfreetype6-dev is already the newest version (2.8.1-2ubuntu2.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
import os
os.environ['QUANDL_API_KEY'] = '1MVxkHJfSsgBjS2KYERG'


from zipline.utils.run_algo import load_extensions

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)

In [ ]:
%load_ext zipline
#Second line to register the magic

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [ ]:
! zipline ingest -b quantopian-quandl

[2021-06-12 12:25:03.583417] INFO: zipline.data.bundles.core: Ingesting quantopian-quandl.
[2021-06-12 12:25:05.685069] INFO: zipline.data.bundles.quandl: Writing data to /root/.zipline/data/quantopian-quandl/2021-06-12T12;25;03.177554.


In [ ]:
#Check existing data
!zipline bundles

csvdir <no ingestions>
quandl <no ingestions>
quantopian-quandl 2021-06-12 12:25:03.177554
quantopian-quandl 2021-06-12 09:06:12.571332


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SHFE-SNX2020.csv to SHFE-SNX2020 (2).csv


In [ ]:
#Get correct trading calendar days
#%%
import pandas as pd
import csv
import numpy as np
from trading_calendars import get_calendar
from zipline.utils.calendars import TradingCalendar
from zipline.utils.tradingcalendar import get_trading_days
from pandas import Timestamp


#%%
#This is the process to get the correct trading days, which will then be use in the Bundle Ingest process to index the days
trading_days = get_trading_days(start=Timestamp('2019-11-18 00:00:00+0000', tz='UTC'), 
                                end=Timestamp('2020-11-16 02:21:29.456875+0000')
                                ).date.astype(str)

# %%
# Delete any specific days that give errors
trading_days = np.delete(trading_days, np.where(trading_days == '2018-12-05'))
trading_days[trading_days == '2018-12-05']


#%%
# Create a CSV file to set the Zipline Bundle Ingest process against
#with open('trading_calendar.csv', 'w', newline='') as f:
with open('trading_calendar.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(trading_days)

tc = pd.read_csv('trading_calendar.csv')
tc.head()

,2019-11-18,2019-11-19,2019-11-20,2019-11-21,2019-11-22,2019-11-25,2019-11-26,2019-11-27,2019-11-29,2019-12-02,...,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-16


In [ ]:
#Prepare your bundle
#%%
import csv
import numpy as np
import pandas as pd
import os



with open('trading_calendar.csv') as f:
    reader = csv.reader(f)
    data = list(reader)

arr = np.array(data)
trading_days = arr.ravel()


def format_bundle(file, outdir):
       
    count = 0
    #ie read only for f ie for dates that are the same as in trading calendar
    for f in ['SHFE-SNX2020.csv']:  
        
      
        df = pd.read_csv(io.BytesIO(uploaded['SHFE-SNX2020.csv']), index_col=0, parse_dates=['Date'])
        #df = pd.read_csv('{}/{}'.format(indir, f), index_col='date')
        #df = pd.read_csv('{}/{}'.format(file, f), index_col='Date')
        df = df.reindex(trading_days)

    
        df.reset_index(inplace=True)
        
        # Check if there is there is any divident, if not make it zero
        if not 'dividend' in df.columns:
            df['dividend'] = 0.0
        
        df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'dividend']]
        df['dividend'].fillna(0.00, inplace=True)
        df['ratio'] = 1 
        
        df.rename(columns={'Open': 'open',
                           'High': 'high',
                           'Low': 'low',
                           'Close': 'close',
                           'Volume': 'volume'
                           }, inplace=True)
        

        df = df.round({'open':2,
                  'high':2,
                  'low':2,
                  'close':2,
                  'volume':1,
                  'dividend':2})

        df.to_csv('abc.csv',sep=',')
        #df.to_csv('{}/{}'.format(u'/content/', f), index=False)

        count += 1
        
    return ('{} files was adjusted'.format(count))


#%%  Execute the function to format the files
format_bundle('SHFE-SNX2020.csv', u'/content/mainfile')


'1 files was adjusted'

In [ ]:
%pwd


u'/content'

In [ ]:
from zipline.data.bundles import register
from zipline.data.bundles.csvdir import csvdir_equities

# Set the start and end dates of the bars, should also align with the Trading Calendar
start_session = pd.Timestamp('2019-11-18', tz='utc')
end_session = pd.Timestamp('2020-11-16', tz='utc')



register(
    'custom-bundle',   # What to call the new bundle
    csvdir_equities(
        u'/content/mainfile/',  # Directory where the formatted bar data is
    ),
    calendar_name='SSE', # Shanghai equities default
    start_session=start_session,
    end_session=end_session
)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Overwriting bundle with name 'custom-bundle'


<bound method CSVDIRBundle.ingest of <zipline.data.bundles.csvdir.CSVDIRBundle instance at 0x7f98bb85eaa0>>

In [ ]:
! zipline ingest -b 'custom-bundle'

WE COULD ALSO HAVE CREATED JUST A DATA FRAME OR PANEL TO WORK UPON, IF THE DESIRE IF TO WORK WITH FEW CSVs

In [ ]:
import pandas as pd
import io
from collections import OrderedDict
import pytz

df = OrderedDict()
df['SHFE'] = pd.read_csv(io.BytesIO(uploaded['SHFE-SNX2020.csv']), index_col=0, parse_dates=['Date'])
df['SHFE'] = df['SHFE'][["Open" , "High" ,"Low" , "Close", "Volume" ]]
print(df['SHFE'].head())


                Open      High       Low   Close  Volume
Date                                                    
2019-11-18  146000.0  147250.0  146000.0  147250      26
2019-11-19  144800.0  145900.0  144800.0  145900      12
2019-11-20  143010.0  145600.0  143010.0  144660      10
2019-11-21  145310.0  145310.0  145300.0  145300      12
2019-11-22  146100.0  146140.0  145300.0  145300      16


In [ ]:
import pytz
panel = pd.Panel(df)
panel.minor_axis = ["Open" , "High" ,"Low" , "Close", "Volume" ]
panel.major_axis = panel.major_axis.tz_localize(pytz.utc)
print(panel)

<class 'pandas.core.panel.Panel'>
Dimensions: 1 (items) x 242 (major_axis) x 5 (minor_axis)
Items axis: SHFE to SHFE
Major_axis axis: 2019-11-18 00:00:00+00:00 to 2020-11-16 00:00:00+00:00
Minor_axis axis: Open to Volume


In [ ]:
bundle_name = panel
from zipline.data.bundles import register
from zipline.data.bundles.csvdir import csvdir_equities
injest_func = csvdir_equities(['daily'], bundle_name)

csvdir <no ingestions>
quandl <no ingestions>
quantopian-quandl 2021-06-12 12:25:03.177554
quantopian-quandl 2021-06-12 09:06:12.571332
